### Data Prep

Write a script to transform input CSV to desired output CSV. 

You will find a CSV file in the files folder under [data.csv](files/data.csv). There are two steps (plus an optional bonus - Date offset) to this part of the test. Each step concerns manipulating the values for a single field according to the step's requirements. The steps are as follows:

**String cleaning** - The bio field contains text with arbitrary padding, spacing and line breaks. Normalize these values to a space-delimited string.

**Code swap** - There is a supplementary CSV in the files folder under [state_abbreviations.csv](files/state_abbreviations.csv). This "data dictionary" contains state abbreviations alongside state names. For the state field of the input CSV, replace each state abbreviation with its associated state name from the data dictionary.

(Optional) **Date offset** - The start_date field contains data in a variety of formats. These may include e.g., "June 23, 1912" or "5/11/1930" (month, day, year). But not all values are valid dates. Invalid dates may include e.g., "June 2018", "3/06" (incomplete dates) or even arbitrary natural language. Add a start_date_description field adjacent to the start_date column to filter invalid date values into. Normalize all valid date values in start_date to ISO 8601 (i.e., YYYY-MM-DD).

Your script should take [data.csv](files/data.csv) as input and produce a cleansed "enriched.csv" file according to the step requirements above. 

_Please commit a _enriched.csv_ file along with your solution code in the `solution/csv` folder._



In [19]:
import sys
import csv
import pandas as pd
from dateutil import parser
from dateutil.parser import isoparse
import datetime
import re


data = pd.read_csv('../../files/data.csv')
state = pd.read_csv('../../files/state_abbreviations.csv')
data['start_date_description']="na"

for index, row in data.iterrows():
# String cleaning
    data.at[index,'bio'] = re.sub( '\s+', ' ', row['bio'] ).strip()
# Code Swap
    data.at[index,'state'] = state.loc[state['state_abbr'] == row['state']].state_name.to_string().split()[1]
# Date Offset
    try:
        d = parser.parse(row['start_date'])
        data.at[index,'start_date_description'] = d.strftime("%Y-%m-%d")
    except ValueError:
        data.at[index,'start_date_description'] = "Invalid Date"
    if len(row['start_date'].split()) == 2:
        data.at[index,'start_date_description'] = "Invalid Date"
    if len(row['start_date']) < 8:
        data.at[index,'start_date_description'] = "Invalid Date"
        
data.to_csv('enriched.csv')
data.head(100)


name gender   birthdate  \
0                  Leslee Corwin      M  1974-02-01   
1                  Orris Kuvalis      M  1997-01-08   
2                   Afton Hirthe      M  1970-08-25   
3                  Olinda Wisoky      F  2007-01-11   
4       Dr. Annmarie Schmitt PhD      M  1995-09-29   
5                   Mathew Grady      F  1973-04-30   
6               Ailene Abernathy      F  2002-06-19   
7                 Justen Carroll      M  2003-08-27   
8                  Nikita Torphy      M  1979-08-02   
9              Murry Waelchi PhD      F  2013-12-03   
10               Darrick Schultz      F  1987-04-29   
11  Mrs. Charlize Williamson DVM      F  1997-02-13   
12               Brett Hettinger      F  1986-03-27   
13                   Benito Wolf      F  1990-05-15   
14                  Aloys Parker      M  1979-10-10   
15              Pallie Reilly MD      M  2012-10-27   
16            Francisquita Boehm      M  2004-12-05   
17               Colonel Johnson      F  1987-05-11   
18                 Marina Kemmer      F  1979-06-14   
19            Miss Effie Hermann      M  1982-02-09   
20                Wendi Schinner      F  1977-12-02   
21               Dr. Liam Fisher      F  1997-09-25   
22                 Muriel Brakus      F  2006-12-14   
23                 Fredy Spencer      M  2008-11-12   
24             Severo Jakubowski      M  1977-07-02   
25          Miss Zona Beatty DDS      M  1996-08-23   
26   Dr. Kathrine Stoltenberg MD      M  1980-07-16   
27             Skyla Oberbrunner      F  1996-09-25   
28                   Maci Stokes      F  2000-01-24   
29                   Rheta Ebert      M  1990-08-17   
..                           ...    ...         ...   
70           Ms. Rowena Franecki      F  1985-03-18   
71                Isidro Farrell      F  1991-06-29   
72        Dr. Rae Schowalter PhD      F  2000-08-23   
73           Catalina Balistreri      M  1978-04-10   
74                 Jeana Dickens      M  1980-08-10   
75                   Harlow Mraz      M  1999-09-30   
76                Antwon Osinski      F  1997-02-06   
77            Arielle Altenwerth      M  1973-07-04   
78                  Tatiana Mraz      M  2006-06-13   
79                     Dema Lind      F  1999-02-12   
80           Dr. Cleo Blanda DVM      M  1977-08-23   
81               Braydon Collier      M  2002-02-26   
82             Bertina Lueilwitz      M  1979-07-23   
83             Pranav Wuckert II      F  1976-05-07   
84               Shelbi Mosciski      M  1993-08-10   
85             Georgine Yost PhD      F  1999-08-21   
86                 Domenica King      M  1974-12-18   
87                  Hung Kessler      M  1985-06-08   
88                   Falon Kunde      F  1998-03-04   
89           Ms. Sharron Shields      F  2005-08-24   
90               Guillermo Haley      F  2006-02-28   
91         Wendell Runolfsdottir      M  2000-05-14   
92          Mr. Armond Purdy DVM      F  2004-09-05   
93               Marcello Von IV      M  1982-02-17   
94        Mr. Jamar Dickinson IV      M  1974-01-13   
95               Octavie DuBuque      M  1989-12-27   
96          Augustina Gibson PhD      M  1973-05-07   
97           Dr. Eino Schaefer I      F  1980-12-29   
98                  Dwayne Lynch      M  2013-01-10   
99                  Pearley Wiza      M  1996-08-25   

                               address                 city         state  \
0                     4933 Weber Walks           Lake Carey        Kansas   
1                      092 Kanye Forge    South Doshiamouth     Tennessee   
2      355 Shaquille Centers Suite 834         Lorriborough      Oklahoma   
3                48328 Rudolph Harbors            Braunport      Nebraska   
4   47861 Satterfield Meadow Suite 420       Bergstromshire         North   
5          59765 Berge Coves Suite 085      Port Jarretbury         South   
6            26158 Tea Crest Suite 735        Ivannachester     Minnesota   
7